In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import csv
import os
from os import listdir
from numpy import genfromtxt, savetxt
plt.rcParams.update({'font.size': 12})

In [ ]:
tab_f = np.arange(20)
tab_attack = ['SF', 'FOE', 'ALIE', 'mimic']
tab_attack_name = ['SF', 'FOE', 'ALIE', 'mimic']
tab_sign = ['-', '--', '-.', ':', 'dashdotdotted' '-', '-', '-']
colors = [(0, 0.4470, 0.7410), (0.8500, 0.3250, 0.0980), (0.4660, 0.6740, 0.1880), (60/255,60/255, 60/255)]
markers = ['v','^','<','>','s','+', 'o']
nb_runs = 5
steps = 10
T = 40001

tab_losses = np.zeros((len(tab_f), len(tab_attack), T))
for i, f in enumerate(tab_f):
    for j, attack in enumerate(tab_attack):
        if f == 0 and j > 0:
            tab_losses[i][j] = tab_losses[i][0]
        else :
            tab_losses[i][j] = genfromtxt('./save_fig4/Losses_'+str(f)+'_'+attack+'.csv', delimiter=',')
#         print(f, attack)


In [ ]:
def grad_function(theta, X, y, lbda):
    return np.reshape((X.T @ X /len(X) + config['lbda'] * np.eye(d)) @  theta, (-1,1)) - (X.T @ np.reshape(y, (-1,1)))/len(X)

In [ ]:
theta_star = genfromtxt('./save_fig4/theta_star.csv', delimiter=',')

sum_norm_theta = genfromtxt('./save_fig4/sum_norm_theta_star.csv', delimiter=',')

loss_star = genfromtxt('./save_fig4/loss_star.csv', delimiter=',')
L_mu = genfromtxt('./save_fig4/L_mu.csv', delimiter=',')
kappas = genfromtxt('./save_fig4/Kappas_.csv', delimiter=',')

L = L_mu[0]
mu = L_mu[1]

print(L, mu)

tab_RHS = []
L_0 = tab_losses[0,0,0]
for f in range(20):
    n = 20 + f
    kappa = f/(n-2*f)
    RHS = (3 * kappas[f] / mu) * sum_norm_theta + np.exp(-(mu/(3*L))*T) * (L_0 - loss_star)
    tab_RHS.append(RHS)

In [ ]:
frac = np.arange(1,20)/np.arange(21,40)


fig, axes = plt.subplots()
for b in range(len(tab_attack)):
    axes.plot(frac, np.min(tab_losses[:,b, :], axis = 1)[1:]-loss_star, label=tab_attack[b], linestyle = tab_sign[b], linewidth=2, color = colors[b])
axes.plot(frac, tab_RHS[1:], linewidth=2, linestyle = (0, (3, 1, 1, 1, 1, 1)) , label='Our upper bound', color = 'orange')
plt.grid()

lines = axes.get_lines()
legend1 = plt.legend([lines[i] for i in [0,1,2,3]], tab_attack_name, loc='best')
legend2 = plt.legend([lines[i] for i in [4]], ['Our upper bound'], loc='best')
axes.add_artist(legend1)
axes.add_artist(legend2)

plt.xlabel('Fraction of Byzantine workers')
plt.ylabel('Best error gap over 40000 steps')
plt.yscale('log')
plt.xlim(frac[0],frac[-1])
plt.savefig('tmp.pdf')